<h1><center>Laboratorio 6: Optimización de modelos 🧪</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Otoño 2025</strong></center>

### Cuerpo Docente:

- Profesores: Stefano Schiappacasse, Sebastián Tinoco
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Angelo Muñoz, Valentina Zúñiga

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Ignacio Negrete
- Nombre de alumno 2: Camila Salas


### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/ignacio3645/MDS7202.git)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.


### Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [79]:
import pandas as pd

df = pd.read_csv('sales.csv')

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [80]:
from sklearn import set_config
from sklearn.model_selection import train_test_split

set_config(transform_output="pandas")

# Se separan los datos en train, validation y test
X, X_test, y, y_test = train_test_split(df, df['quantity'], test_size=0.1, random_state=42, shuffle=True) # se hace de manera aleatoria, manteniendo la distribucion original

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [4]:
from sklearn.preprocessing import FunctionTransformer
import datetime

# Se implementa function transformer
# Se crea funciona a aplicar
def date(fecha):
    # Se separa por dia, mes año, utilizando datetime.datetime.stptime y guardando en formato categorical
    day = pd.DataFrame(fecha.apply(lambda x: datetime.datetime.strptime(x, "%d/%m/%y").day), dtype='category')
    month = pd.DataFrame(fecha.apply(lambda x: datetime.datetime.strptime(x, "%d/%m/%y").month), dtype='category')
    year = pd.DataFrame(fecha.apply(lambda x: datetime.datetime.strptime(x, "%d/%m/%y").year), dtype='category')
    return day, month, year

# Se crea la transformacion
transformer = FunctionTransformer(date)

# Se aplica y se obtienen los dias, meses y años separados
days, months, years = transformer.transform(X_train['date'])

C:\Users\Camila\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_function_transformer.py:318: UserWarning: When `set_output` is configured to be 'pandas', `func` should return a pandas DataFrame to follow the `set_output` API  or `feature_names_out` should be defined.
  warnings.warn(warn_msg.format("pandas"))


In [ ]:
X_train = X_train.drop(columns=['date', 'id', 'quantity'])

# Se agregan los dias, meses y años a x_train
X_train['day'] = days
X_train['month'] = months
X_train['year'] = years
X_train

,city,lat,long,pop,shop,brand,container,capacity,price,day,month,year
4164,Thessaloniki,40.64361,22.93086,351650,shop_4,gazoza,plastic,1.5lt,0.98,30,11,2015
5470,Larisa,39.63689,22.41761,144302,shop_5,gazoza,glass,500ml,0.79,28,2,2017
1527,Irakleion,35.32787,25.14341,135432,shop_2,orange-power,can,330ml,0.55,30,6,2013
6319,Patra,38.24444,21.73444,168501,shop_6,lemon-boost,glass,500ml,0.88,31,12,2017
6455,Patra,38.24444,21.73444,168034,shop_6,lemon-boost,can,330ml,0.55,31,1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...
4715,Irakleion,35.32787,25.14341,137302,shop_2,adult-cola,can,330ml,0.65,30,6,2016
2051,Irakleion,35.32787,25.14341,135432,shop_2,kinder-cola,plastic,1.5lt,3.52,31,12,2013
4566,Thessaloniki,40.64361,22.93086,351702,shop_4,orange-power,can,330ml,0.51,30,4,2016
7312,Thessaloniki,40.64361,22.93086,354290,shop_4,kinder-cola,can,330ml,0.68,30,11,2018


In [ ]:
# Se aplica y se obtienen los dias, meses y años separados
days_val, months_val, years_val = transformer.transform(X_val['date'])
X_val = X_val.drop(columns=['date', 'id', 'quantity']) 

# Se agregan los dias, meses y años a x_val
X_val['day'] = days_val
X_val['month'] = months_val
X_val['year'] = years_val
X_val

C:\Users\Camila\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_function_transformer.py:318: UserWarning:

When `set_output` is configured to be 'pandas', `func` should return a pandas DataFrame to follow the `set_output` API  or `feature_names_out` should be defined.



,city,lat,long,pop,shop,brand,container,capacity,price,day,month,year
2624,Athens,37.96245,23.68708,668203,shop_3,kinder-cola,can,330ml,0.85,30,6,2014
5060,Athens,37.97945,23.71622,665102,shop_1,lemon-boost,glass,500ml,0.84,31,10,2016
5054,Athens,37.97945,23.71622,665102,shop_1,adult-cola,glass,500ml,1.18,31,10,2016
4274,Irakleion,35.32787,25.14341,137302,shop_2,adult-cola,can,330ml,0.71,31,1,2016
3533,Irakleion,35.32787,25.14341,138560,shop_2,lemon-boost,can,330ml,0.72,30,4,2015
...,...,...,...,...,...,...,...,...,...,...,...,...
5037,Irakleion,35.32787,25.14341,137302,shop_2,adult-cola,plastic,1.5lt,2.66,30,9,2016
1686,Athens,37.97945,23.71622,671022,shop_1,kinder-cola,glass,500ml,1.14,31,8,2013
2836,Athens,37.96245,23.68708,668203,shop_3,adult-cola,plastic,1.5lt,3.25,31,8,2014
131,Thessaloniki,40.64361,22.93086,346502,shop_4,gazoza,glass,500ml,0.61,29,2,2012


In [ ]:
# Se aplica y se obtienen los dias, meses y años separados
days_test, months_test, years_test = transformer.transform(X_test['date'])
X_test = X_test.drop(columns=['date', 'id', 'quantity'])

# Se agregan los dias, meses y años a x_test
X_test['day'] = days_test
X_test['month'] = months_test
X_test['year'] = years_test
X_test

C:\Users\Camila\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_function_transformer.py:318: UserWarning:

When `set_output` is configured to be 'pandas', `func` should return a pandas DataFrame to follow the `set_output` API  or `feature_names_out` should be defined.



,city,lat,long,pop,shop,brand,container,capacity,price,day,month,year
5671,Athens,37.97945,23.71622,665871,shop_1,adult-cola,glass,500ml,1.27,31,5,2017
2177,Athens,37.96245,23.68708,668203,shop_3,adult-cola,can,330ml,0.31,31,1,2014
1142,Patra,38.24444,21.73444,166301,shop_6,lemon-boost,plastic,1.5lt,1.78,31,1,2013
6648,Athens,37.97945,23.71622,664046,shop_1,adult-cola,glass,500ml,1.01,30,4,2018
3458,Athens,37.97945,23.71622,667237,shop_1,kinder-cola,glass,500ml,1.36,30,4,2015
...,...,...,...,...,...,...,...,...,...,...,...,...
4333,Irakleion,35.32787,25.14341,137302,shop_2,orange-power,can,330ml,0.58,31,1,2016
573,Thessaloniki,40.64361,22.93086,346502,shop_4,adult-cola,can,330ml,0.76,31,7,2012
932,Thessaloniki,40.64361,22.93086,346502,shop_4,orange-power,glass,500ml,0.64,30,11,2012
6749,Irakleion,35.32787,25.14341,137154,shop_2,adult-cola,glass,500ml,1.14,31,5,2018


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

# Se definen columnas categoricas
col_categoricas = ['day', 'month', 'year', 'city', 'shop', 'brand', 'container', 'capacity'] 

# Se definen columnas numericas
col_numericas = X_train.columns.drop(col_categoricas).to_list()

# se define la transformacion a las variables categoricas
trans_categoricas = Pipeline([('onehotencoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

# se define la transformacion a las variables numericas
trans_numericas = Pipeline([('standardscaler', StandardScaler())])

# se define el column transformer
column_transformer = ColumnTransformer([('numericas', trans_numericas, col_numericas),
                                        ('categoricas',trans_categoricas, col_categoricas)],
                                        verbose_feature_names_out = False)

column_transformer.set_output(transform='pandas')

X_train_trans = column_transformer.fit_transform(X_train)
X_train_trans

,lat,long,pop,price,day_28,day_29,day_30,day_31,month_1,month_2,...,brand_gazoza,brand_kinder-cola,brand_lemon-boost,brand_orange-power,container_can,container_glass,container_plastic,capacity_1.5lt,capacity_330ml,capacity_500ml
4164,1.407085,-0.309472,-0.008795,-0.265539,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5470,0.798450,-0.782962,-0.903651,-0.498428,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1527,-1.806664,1.731676,-0.941931,-0.792603,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
6319,-0.043386,-1.413208,-0.799215,-0.388112,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
6455,-0.043386,-1.413208,-0.801230,-0.792603,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4715,-1.806664,1.731676,-0.933861,-0.670030,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2051,-1.806664,1.731676,-0.941931,2.847817,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4566,1.407085,-0.309472,-0.008571,-0.841633,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
7312,1.407085,-0.309472,0.002598,-0.633258,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
from sklearn.dummy import DummyRegressor
# Se juntan los pasos en un pipeline y se entrena
pipeline_dummy = Pipeline([('columntransformer', column_transformer),
                     ('dummy', DummyRegressor(strategy='mean'))])

pipeline_dummy.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numericas',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['lat', 'long', 'pop',
                                                   'price']),
                                                 ('categoricas',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['day', 'month', 'year',
                                                   'city', 'shop', 'brand',
                                                   'container', 'capacity'])],
                                   verbose_feature_names_out=False)),
                ('dummy', DummyRegressor())])

In [ ]:
from sklearn.metrics import mean_absolute_error

# se obtiene la prediccion y luego el valor mean absolute error 
y_pred_dummy = pipeline_dummy.predict(X_val)

mae_dummy = mean_absolute_error(y_val, y_pred_dummy)
print('Valor de mean absolute error de DummyRegressor:', mae_dummy)

Valor de mean absolute error de DummyRegressor: 13576.56699051175


**Respuesta 1.5:** el valor de esta métrica en el negocio indica que las diferencias promedios obtenidas de la predicción del modelo dummy con las reales son de cerca de 13576 cantidades, osea está lejos de los valores reales.

In [87]:
import xgboost as xgb
# Se juntan los pasos en un pipeline

pipeline_xgb = Pipeline([('columntransformer', column_transformer),
                     ('xgboost', xgb.XGBRegressor())])

pipeline_xgb.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numericas',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['lat', 'long', 'pop',
                                                   'price']),
                                                 ('categoricas',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['day', 'month', 'year',
                                                   'city', 'shop', 'brand',
                                                   'container', 'capacity'])],
                                   verb...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [ ]:
# se obtiene la prediccion y luego el valor mean absolute error 
y_pred_xgb = pipeline_xgb.predict(X_val)

mae_xgb = mean_absolute_error(y_val, y_pred_xgb)
print('Valor de mean absolute error de XGBRegressor:', mae_xgb)

Valor de mean absolute error de XGBRegressor: 2381.2630062188546


**Respuesta 1.6:** (como cambia el mae, es mejor o peor) en este caso el valor del mae es casi la mitad del mae de dummy, lo que indica que es mejor, ya que hay una menor diferencia entre los valores predichos y los reales.

In [12]:
import pickle
# Se guardan ambos modelos en un archivo pkl

with open('model_dummy.pkl', 'wb') as file:  
    pickle.dump(pipeline_dummy, file)

with open('model_xgbregressor.pkl', 'wb') as file:  
    pickle.dump(pipeline_xgb, file)

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [91]:
# Se vuelve a entrenar el pipeline con XGBRegressor
import xgboost as xgb

pipeline_xgb_mc = Pipeline([('columntransformer', column_transformer),
                     ('xgboost', xgb.XGBRegressor(monotone_constraints = {'price': -1}))])

pipeline_xgb_mc.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numericas',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['lat', 'long', 'pop',
                                                   'price']),
                                                 ('categoricas',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['day', 'month', 'year',
                                                   'city', 'shop', 'brand',
                                                   'container', 'capacity'])],
                                   verb...
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints={'price': -1},
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None, ...))])

In [ ]:
# se obtiene la prediccion y luego el valor mean absolute error 
y_pred_xgb_mc = pipeline_xgb_mc.predict(X_val)

mae_xgb_mc = mean_absolute_error(y_val, y_pred_xgb_mc)
print('Valor de mean absolute error de XGBRegressor:', mae_xgb_mc)

Valor de mean absolute error de XGBRegressor: 2549.220236219699


**Respuesta 2:** al realizar este cambio agregando la monotonía negativa, se puede notar que se obtiene un valor de mae bastante cercano al obtenido con el modelo xgboost anterior, pero con un leve aumento, por lo que empeora un poco el resultado, ya que ahora la diferencia es más grande. Por lo anterior, considerando lo que dijo el amigo como una mejora, entonces no está en lo correcto.

In [93]:
# Se guarda el modelo en un archivo picle
with open('model_xgbregressor_mc.pkl', 'wb') as file:  
    pickle.dump(pipeline_xgb_mc, file)

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [16]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Camila\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [94]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)


def objective(trial):
    # Hiperparámetros que se pide optimizar con sus rangos
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
    n_estimators = trial.suggest_int("n_estimators", 50, 1000)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    max_leaves = trial.suggest_int("max_leaves", 0, 100)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 5)
    reg_alpha = trial.suggest_float("reg_alpha", 0.0, 1.0)
    reg_lambda = trial.suggest_float("reg_lambda", 0.0, 1.0)
    min_freq = trial.suggest_float("min_frequency", 0.0, 1.0)

    # Le asignamos a OneHotEncodoer el parametro optimizado 
    trans_categoricas = Pipeline([
        ('onehotencoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False, min_frequency=min_freq))
    ])

    # Actualizamos el transformer con el encoder actualizado
    column_transformer = ColumnTransformer([
        ('numericas', trans_numericas, col_numericas),
        ('categoricas', trans_categoricas, col_categoricas)
    ], verbose_feature_names_out=False)
    column_transformer.set_output(transform="pandas")

    # Entrenamos con lo optimizado
    model = xgb.XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=42,
        verbosity=0
    )

    # Pipeline con el modelo y transformación
    pipeline = Pipeline([
        ('columntransformer', column_transformer),
        ('xgb', model)
    ])

    pipeline.fit(X_train, y_train)


    y_pred = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)

    # Guardamos
    trial.set_user_attr("best_pipeline", pipeline)

    return mae

In [95]:
#Finamos una semilla para reproducibilidad
sampler = TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)
study.optimize(objective, timeout=300)  # por 5 minutos

In [96]:
print("Número de trials:", len(study.trials))
print("Mejor MAE:", study.best_value)
print("Mejores hiperparámetros:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

Número de trials: 99
Mejor MAE: 1995.3549304399335
Mejores hiperparámetros:
  learning_rate: 0.0968169751057792
  n_estimators: 821
  max_depth: 9
  max_leaves: 64
  min_child_weight: 3
  reg_alpha: 0.8826455784225954
  reg_lambda: 0.4190022297365533
  min_frequency: 0.06567163373537473


**Respuesta 3.3:** 
En cuanto a los resultados del modelo, se puede ver que hubo una mejora pues disminuye el mae. Esto se debe principalmente a que estamos optimizando hiperparámetros clave que nos permiten obtener un mejor rendimiento en el modelo.

**Respuesta 3.4:**

En primer lugar definamos lo que significa cada hiperparámetro

- Learning rate: Es la tasa a la que el modelo itera para encontrar el optimó, menor learning rate implica mayor cantidad de iteraciones para encontrar el óptimo, luego el rango propuesto es el mas común en la literatura.

- n_estimators: Son la cantidad de arboles que constituyen el modelo, mas arboles capturan relaciones complejas, muchos arboles pueden provocar sobreajuste. Por tanto el rango (50,1000) es razonable

- max_depth: Controla la profundidad del árbol, al igual que antes nos enfrentamos al dilema complejidad - sobreajuste. En cuanto al rango propuesto profundidad entre 3 y 10 permite controlar el trade-off y no irse a situaciones extremas. 

- max_leaves: Define el máximo de hojas que puede tener un árbol, nuevamente el dilema complejidad - sobreajuste. Quizas los valores en el rango [0,10] no son muy necesarios. 

- min_child_weight: Controla la cantidad mínima de muestras que debe tener un nodo para que se generen particiones sobre él, es decir controla la profundidad. En cuanto al rango se pudieron explorar valores mas altos, por ejemplo hasta 7-10 para evitar mas sobreajuste

- reg_alpha: Es la regularizacion Lasso, que penaliza la suma de los valores absolutos de los pesos de los nodos de los árboles. El rango es el correcto pues el coeficiente se mueve en el rango [0,1]

- reg_lambda: Es la regularizacion de Ridge, que penaliza el cuadrado de la suma de los pesos. Al igual que Lasso vive en el rango [0,1]

- min_frecuency: Es un umbral mínimo de frecuencia que debe tener una categoria para que el encoder la considere como una columna separada, por tanto ayuda a reducir dimensionalidad. Valores en 0 significa que se codifica todo, y valores en 1 significa que para poder codificarse el 100% de los registros deben pertencer a la misma categoría. Por tanto considerar valores cercanos a 1 no tiene mucho sentido.

In [20]:
# Guardamos el modelo
model_xgb_optuna = study.best_trial.user_attrs["best_pipeline"]

with open("model_xgb_optuna.pkl", "wb") as f:
    pickle.dump(model_xgb_optuna, f)

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [21]:
!pip install optuna-integration[xgboost]


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Camila\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [97]:
from optuna.integration import XGBoostPruningCallback
optuna.logging.set_verbosity(optuna.logging.WARNING)


def objective(trial):
    # Idem a la parte anterior
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
    n_estimators = trial.suggest_int("n_estimators", 50, 1000)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    max_leaves = trial.suggest_int("max_leaves", 0, 100)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 5)
    reg_alpha = trial.suggest_float("reg_alpha", 0.0, 1.0)
    reg_lambda = trial.suggest_float("reg_lambda", 0.0, 1.0)
    min_freq = trial.suggest_float("min_frequency", 0.0, 1.0)

    trans_categoricas = Pipeline([
        ('onehotencoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False, min_frequency=min_freq))
    ])

    column_transformer = ColumnTransformer([
        ('numericas', trans_numericas, col_numericas),
        ('categoricas', trans_categoricas, col_categoricas)
    ], verbose_feature_names_out=False)
    column_transformer.set_output(transform="pandas")

    # modelo con pruning
    model = xgb.XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=42,
        verbosity=0,
        eval_metric='mae'
    )

    pipeline = Pipeline([
        ('columntransformer', column_transformer),
        ('xgb', model)
    ])


    X_train_trans = column_transformer.fit_transform(X_train)
    X_val_trans = column_transformer.transform(X_val)

    # Eval set para pruning
    eval_set = [(X_val_trans, y_val)]

    # Pruning callback
    pruning_callback = XGBoostPruningCallback(trial, "validation_0-mae")

    # Entrenamiento con los parametros vía pipeline de la forma xgb__param = ...

    pipeline.set_params(xgb__early_stopping_rounds= 20,
        xgb__callbacks= [pruning_callback],
        xgb__verbose= False)
    
    pipeline.fit(
        X_train, y_train,
        xgb__eval_set= eval_set
    )


    y_pred = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)


    trial.set_user_attr("best_pipeline", pipeline)

    return mae

In [98]:
# Ahora los resultados con pruning
study_p = optuna.create_study(direction="minimize",sampler=sampler)
study_p.optimize(objective, timeout=300, show_progress_bar=True)

   0%|          | 00:00/05:00

[0]	validation_0-mae:13129.87242
[1]	validation_0-mae:12709.88850
[2]	validation_0-mae:12357.16143
[3]	validation_0-mae:12023.76590
[4]	validation_0-mae:11735.20983
[5]	validation_0-mae:11504.37806
[6]	validation_0-mae:11248.62201
[7]	validation_0-mae:11012.35705
[8]	validation_0-mae:10831.17509
[9]	validation_0-mae:10630.62875
[10]	validation_0-mae:10485.96918
[11]	validation_0-mae:10320.01429
[12]	validation_0-mae:10189.51507
[13]	validation_0-mae:10084.20625
[14]	validation_0-mae:9968.11272
[15]	validation_0-mae:9884.43561
[16]	validation_0-mae:9777.04166
[17]	validation_0-mae:9708.34443
[18]	validation_0-mae:9625.92273
[19]	validation_0-mae:9564.67145
[20]	validation_0-mae:9517.86041
[21]	validation_0-mae:9461.07140
[22]	validation_0-mae:9426.89466
[23]	validation_0-mae:9365.39266
[24]	validation_0-mae:9322.49264
[25]	validation_0-mae:9267.55766
[26]	validation_0-mae:9229.33810
[27]	validation_0-mae:9181.89115
[28]	validation_0-mae:9152.48387
[29]	validation_0-mae:9117.57114
[30]	v

In [99]:
print("Número de trials:", len(study_p.trials))
print("Mejor MAE:", study_p.best_value)
print("Mejores hiperparámetros:")
for key, value in study_p.best_params.items():
    print(f"  {key}: {value}")

Número de trials: 113
Mejor MAE: 2073.4760647587555
Mejores hiperparámetros:
  learning_rate: 0.08748994441762048
  n_estimators: 1000
  max_depth: 6
  max_leaves: 74
  min_child_weight: 2
  reg_alpha: 0.2921478015140797
  reg_lambda: 0.357230313021342
  min_frequency: 0.054516920962849595


**Respuestas:**
En primer lugar, prunning es una técnica que se utiliza en entrenamiento para detener anticipadamente trials que no muestran un buen desempeño. En particular en el codigo anterior se determino que si el modelo no mejoraba su mae dentro de 20 rondas consecutivas, se detiene. Lo esperable con esto es que alcancemos a hacer mas trials.

Luego efectivamente eso es lo observado, en el modelo sin prunning en 5 minutos se alcanzan a hacer 99 trials, sin embargo al hacer detenciones tempranas se lograron hacer 113 trials. Por otra parte el mae es peor en el modelo con prunning, esto se puede deber a que el modelo detiene las trials antes de encontrar su punto óptimo. Otro hiperparámetro que vale la pena mencionar es max_depth, pues al detener anticipadamente los trials el modelo tiende a evitar el sobreajuste, es decir árboles menos profundos.

In [67]:
# Guardamos el modelo
model_xgb_pruning = study_p.best_trial.user_attrs["best_pipeline"]

with open("model_xgb_pruning.pkl", "wb") as f:
    pickle.dump(model_xgb_pruning, f)

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [100]:
from optuna.visualization import plot_optimization_history

# Grafico de historial de optimizacion
plot_optimization_history(study_p)

In [101]:
from optuna.visualization import plot_parallel_coordinate

# Grafico de coordenadas paralelas
plot_parallel_coordinate(study_p)

In [102]:
from optuna.visualization import plot_param_importances

# Grafico de importancia de parametros
plot_param_importances(study_p)

**Respuesta 5.4:** se comienzan a observar mejorar notables en los resultados desde los primeros intentos, pero a medida que se avanza, solo en algunos casos se logra obtener un calce entre el valor objetivo y el mejor valor.

**Respuesta 5.5:** si se observa el gráfico de coordenadas paralelas, se puede notar una tendencia de los mejores resultados en pasar por ciertos valores de hiperparámetros, salvo ciertas excepciones, por lo que a pesar de todos los intentos, al parecer hay un camino que conduce a obtener estos mejores resultados. Esta tendencia se ve con valores bajos para 'objective value', 'min frequency', 'n estimators' y 'reg lambda', y con valores que varian un poco en los demás hiperparámetros, pero que en la mayoría de los casos son más altos en su rango.

**Respuesta 5.6:** observando el gráfico de importancia de parámetros, si se hace un top 3 de hiperparámetros de mayor importancia, se puede notar que estos son 'min_frequency', 'n_estimatos' y 'reg_lambda'. Luego de esos 3, se puede notar la importancia además de los siguientes hiperparámetros, por lo que según la cantidad que se decida utilizar, es importante tener en consideración este orden.

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [103]:
# Se genera una tabla resumen de mae
mae_optuna = study.best_value
mae_prunning = study_p.best_value
tabla = {'mae': ['mae_dummy', 'mae_xgb', 'mae_xgb_mc', 'mae_optuna', 'mae_prunning'],
         'valores': [mae_dummy, mae_xgb, mae_xgb_mc, mae_optuna, mae_prunning]
}

tabla = pd.DataFrame(tabla)
tabla

,mae,valores
0,mae_dummy,13576.566991
1,mae_xgb,2381.263006
2,mae_xgb_mc,2549.220236
3,mae_optuna,1995.354930
4,mae_prunning,2073.476065


**Respuesta 6.2:** si se observa la tabla de los valores MAE, se puede notar que el mejor modelo es el obtenido utilizando optuna, con el menor valor de mae, por lo que la diferencia entre los valores predichos y reales son de tan solo 1995 cantidades.

In [104]:
# Se predice sobre el conjunto test para el mejor modelo
y_pred_bm= model_xgb_optuna.predict(X_test) #y_pred_best_model

# Se obtiene el MAE
mae_bm = mean_absolute_error(y_test, y_pred_bm)
print('Valor de mean absolute error de XGBRegressor:', mae_bm)

Valor de mean absolute error de XGBRegressor: 2200.7976846567108


**Respuesta 6.4:** Al comparar las métricas del mejor modelo obtenidas entre las predicciones sobre el conjunto de validación y test, se puede notar que efectivamente cambian, con un empeoramiento, lo que se puede ver en el valor de MAE obtenido al utilizar el conjunto test. Lo anterior ocurre porque además de tener datos diferentes a los de X_val, se tiene una menor cantidad de estos, por lo que es posible obtener diferencias. Ademas optuna optimiza sobre el conjunto de validación lo que puede generar que el modelo se sobreajuste levemente

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://i.pinimg.com/originals/55/3d/42/553d42bea9b10e0662a05aa8726fc7f4.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>